In [7]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras_tqdm import TQDMNotebookCallback
import keras.callbacks

from sqlalchemy import create_engine
import json
import pickle
from itertools import combinations
from operator import itemgetter

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# R
# G
# H

category = 'R'
# category = 'G'
# category = 'H'

In [3]:
engine = create_engine('sqlite:///tab.db')
conn = engine.connect()
conn
dfoo = pd.read_sql_table('race', conn)
print('{} races loaded!'.format(len(dfoo)))

13419 races loaded!


In [4]:
dfo = dfoo.loc[dfoo['race_type'].isin([category])]
print('{} {} races!'.format(len(dfo), category))

4467 R races!


In [5]:
dfo['results'] = dfo['results_data'].map(json.loads)
dfo['runners'] = dfo['runners_data'].map(json.loads)
dfo.head()

/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,id,meeting_name,location,venue_mnemonic,race_type,meeting_date,race_number,race_name,race_start_time,race_status,race_distance,results_data,num_runners,runners_data,quinella,exacta,trifecta,first_four,results,runners
1059,1610,RANDWICK,NSW,S,R,2017-09-16,1,TAB HIGHWAY HANDICAP,2017-09-16 02:15:00,Paying,1000,"[[4], [10], [2], [5]]",NaN,"[{""runnerName"": ""AZZIE'S READY"", ""runnerNumber...",605.4,NaN,NaN,NaN,"[[4], [10], [2], [5]]","[{'win_rank': 4, 'runnerName': 'AZZIE'S READY'..."
1060,1611,RANDWICK,NSW,S,R,2017-09-16,2,SHOOT OUT MILE,2017-09-16 02:50:00,Paying,1600,"[[3], [5], [8], [7]]",NaN,"[{""runnerName"": ""GOLD HORIZON"", ""runnerNumber""...",2296.9,NaN,NaN,NaN,"[[3], [5], [8], [7]]","[{'win_rank': 10, 'runnerName': 'GOLD HORIZON'..."
1061,1612,RANDWICK,NSW,S,R,2017-09-16,3,SCHWEPPES HERITAGE STAKES,2017-09-16 03:25:00,Paying,1100,"[[2], [1], [8], [6]]",NaN,"[{""runnerName"": ""LUCKY LOUIE"", ""runnerNumber"":...",1421.1,NaN,NaN,NaN,"[[2], [1], [8], [6]]","[{'win_rank': 10, 'runnerName': 'LUCKY LOUIE',..."
1062,1613,RANDWICK,NSW,S,R,2017-09-16,4,BILL RITCHIE HANDICAP,2017-09-16 04:00:00,Paying,1400,"[[7], [3], [4], [6]]",NaN,"[{""runnerName"": ""PAJARO"", ""runnerNumber"": 6, ""...",111.9,NaN,NaN,NaN,"[[7], [3], [4], [6]]","[{'win_rank': 9, 'runnerName': 'PAJARO', 'pari..."
1063,1614,RANDWICK,NSW,S,R,2017-09-16,5,MCGRATH ESTATE TEA ROSE STAKES,2017-09-16 04:35:00,Paying,1400,"[[6], [2], [5], [1]]",NaN,"[{""runnerName"": ""PANDEMONIUM"", ""runnerNumber"":...",911.7,NaN,NaN,NaN,"[[6], [2], [5], [1]]","[{'win_rank': 3, 'runnerName': 'PANDEMONIUM', ..."


In [28]:
print('running...')

# create quinella pairs
data_all = pd.DataFrame(columns=['num_runners', 'alpha_win_perc'])
for ri, race in dfo.iterrows():
#     print(race)
    if ri % 100 == 0:
        print('{:.0f}%'.format(ri / len(dfo) * 100)

    # get num runners (requires at least 8)
    num_runners = len([r for r in race['runners'] if r['win_odds'] and r['place_odds']])
    if num_runners < 8:
        continue
    
#     print(race['results'])
    winners = [n[:1] for n in race['results'][:2]]
#     print('winners = {}'.format(winners))

    # data rows will be permutations of 2
    # but combinations if sorted
    runners = sorted(race['runners'], key=itemgetter('win_scaled'))
    combs = combinations(runners, 2)
    
    for comb in combs:
#         print(comb)
#         print('{} {}'.format(comb[0]['runnerNumber'], comb[1]['runnerNumber']))
        row = {
            'num_runners': num_runners,
            'alpha_win_perc': comb[0]['win_perc'],
            'alpha_win_scaled': comb[0]['win_odds'],
            'alpha_win_rank': comb[0]['win_rank'],
            'beta_win_perc': comb[1]['win_perc'],
            'beta_win_scaled': comb[1]['win_odds'],
            'beta_win_rank': comb[1]['win_rank'],
            'outcome': 1 if comb[0]['runnerNumber'] in winners and comb[1]['runnerNumber'] in winners else 0
        }

        data_all = data_all.append([row])
        
#         if row['outcome']:
#             print(row)
#         print('row: {}'.format(row))
#         break
#     break

print('done')

running...
1100
1200
1300
1400
1500
1700
2000
2200
2300
2600
2700
3400
3600
3800
3900
4800
5100
5700
5800
6000
6300
6600
6700
7400
8700
9000
9200
9600
9800
9900
10000
10500
11100
11500
12100
12900
13000
13100
13400
done


In [29]:
data_all.tail(5)

,alpha_win_perc,alpha_win_rank,alpha_win_scaled,beta_win_perc,beta_win_rank,beta_win_scaled,num_runners,outcome
0,0.133333,4.0,7.5,0.200000,2.0,5.0,15,0.0
0,0.133333,4.0,7.5,0.227273,1.0,4.4,15,0.0
0,0.153846,3.0,6.5,0.200000,2.0,5.0,15,0.0
0,0.153846,3.0,6.5,0.227273,1.0,4.4,15,0.0
0,0.200000,2.0,5.0,0.227273,1.0,4.4,15,0.0


In [ ]:
# drop scratched
data = data_pre.dropna(subset=['win_odds', 'place_odds', 'quinella'])
data = data[(data.win_odds > 0)]
data = data[(data.quinella > 0)]
data.describe()

In [ ]:
# get label data
Y = data['finishingPosition']

if bet_type == 'W':
    Y = (Y == 1)
elif bet_type == 'P':
    Y = (Y == 1) | (Y == 2) | ((Y == 3) & (data['num_runners'] >= 8))

#print(data['finishingPosition'].head(10))
Y = Y.astype(int)
# Y.head(10)
Y.describe()

In [ ]:
# xp = fixed odds perc
xp = data['win_perc']
xp.describe()

In [ ]:
# xs = odds scaled
xs = data['win_scaled']
xs.describe()

In [ ]:
# xr = win rank
xr = data['win_rank']
xr.describe()

In [ ]:
# xn = get num runners
xn = data['num_runners']
xn.describe()

In [ ]:
X = pd.concat([xp, xs, xr, xn], axis=1)
X.describe()
# sns.pairplot(pd.concat([X, Y], axis=1))

In [ ]:
# For a single-input model with 2 classes (binary classification):

n = len(X.columns)
print('input dimension = {}'.format(n))

epochs = 1000
print('epochs = {}'.format(epochs))

tag = '{}30x30{}'.format(category, bet_type)
print('tag = {}'.format(tag))
file_name = '/Users/jaco/code/tabby/models/{}.h5'.format(tag)

try:
    model = load_model(file_name)
except OSError:
    model = Sequential()
    model.add(Dense(30, activation='relu', input_dim=n))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

In [ ]:
# Train the model, iterating on the data in batches of 32 samples
tqdm = TQDMNotebookCallback()

tbCallBack = keras.callbacks.TensorBoard(
    log_dir='/Users/jaco/code/tabby/summary/{}'.format(tag), 
    histogram_freq=0,
    write_graph=True,
    write_images=True)

model.fit(
    X.as_matrix(), 
    Y.as_matrix(),
    validation_split=0.2,
    shuffle=True,
    epochs=epochs,
    batch_size=32,
    verbose=0,
    callbacks=[tqdm, tbCallBack])

# creates a HDF5 file 'my_model.h5'
model.save(file_name)